In [2]:
from f_sampling import *
import importlib
from datasets import load_dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import json

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


C:\Users\ajule\AppData\Roaming\Python\Python39\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [ ]:
# for reloading specific modules after change
importlib.reload(f_sampling)

<module 'f_sampling' from 'c:\\Users\\ajule\\OneDrive - McGill University\\Documents\\FATE-SUMM\\Tests\\f_sampling.py'>

# DATASETS AND MODELS


In [3]:
# CNN/DM -> news
# Format: split (train,test,validation), features (article, highlights)
cnn_dm = load_dataset('cnn_dailymail', '3.0.0')
cnn_dm = cnn_dm['validation']['article'] + cnn_dm['test']['article']

model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn', truncation=True)
BartL_cnndm = pipeline('summarization', model=model, tokenizer=tokenizer)

Peg_cnndm = pipeline('summarization', model=AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail"), tokenizer=AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail"))

# Xsum -> news
xsum = load_dataset('xsum')
xsum = xsum['validation']['document'] + xsum['test']['document']

Peg_xsum = pipeline('summarization', model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum"), tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum"))

Reusing dataset cnn_dailymail (C:\Users\ajule\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Using custom data configuration default
Reusing dataset xsum (C:\Users\ajule\.cache\huggingface\datasets\xsum\default\1.2.0\4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [4]:
l = sum_sample(xsum, Peg_cnndm, 2000, res_errors=True, res_time=True)
fsave(f"xsum-pegasus-{len(l)}-s", l)

C:\Python39\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 1024). Running this sequence through the model will result in indexing errors
Your max_length is set to 128, but you input_length is only 95. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 128, but you input_length is only 50. You might consider decreasing max_length manually, e.g. summarizer('.

Total errors: 147.
Total time: 586.95 minutes.


# Processing of pre-summarized datasets

In [3]:
# PRESUMM
# transform into same format as others: {text, summary_text, ref}
res = []
with open('..\Pre_outputs\PreSumm\CNNDM_BertSumExtAbs.jsonl', 'r') as f:
    for line in f:
        l = json.loads(line)
        res.append({"text": l["article"], "summary_text": l["decoded"], "ref" : l["reference"]})
        
fsave("cnndm-peresummext-all", res)

In [54]:
# MATCHSUM
# no article, only summary and ref present...

{'text': "the inaugural event - which is running until 5 november - features scores of local , national and international comedians , with tom stade among the headliners for the opening night . created by aberdeen inspired and breakneck comedy , the festival is using more than 20 city centre venues . there will also be events aimed at children , and comedy workshops . it is hoped it will help put the aberdeen comedy circuit on the map . organiser naz hussain told bbc scotland : `` people are really looking forward to the festival . `` it 's something that will potentially grow into something really big . ''", 'summary_text': "an aberdeen international comedy festival is to be held next year , organisers have announced . 's event .", 'ref': 'a new comedy festival for aberdeen is getting under way .'}


# Keyword filtering

In [ ]:
ewl = ['race', 'black', 'african-american', 'arab', 'mexican', 'asian', 'african', 
'korean', 'japanese', 'chinese', 'china', 'korea', 'africa', 'japan', 'mexico', 'nigger', 'nigga', 'terrorist', 'islam', 'hindu', 'india', 
'aryan, blackness, caucasian, coloured, ethnic, minority, multiracial, racial, PoC, whiteness, nazi, jew, jewism, Hitler, slavery, oppression, supremacism, racism, arab, nationalism, zionism, africa, Middle East, Middle-East,'] 
s = nlp(",".join(ewl))
keywords = []
for token in s:
    if token.is_punct:
        continue
    lem = token.lemma_
    if lem not in keywords:
        keywords.append(lem)

#print(a[0]["text"])
k = keyfilter(keywords, a[:240])
print(len(k))
x = fopen("xsum-pegasus-668-s")
w = keyfilter(keywords, x[:240])

47
